In [2]:
!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 10  908k   10 94599    0     0   107k      0  0:00:08 --:--:--  0:00:08  107k
 48  908k   48  440k    0     0   240k      0  0:00:03  0:00:01  0:00:02  240k
 65  908k   65  591k    0     0   208k      0  0:00:04  0:00:02  0:00:02  208k
 81  908k   81  741k    0     0   192k      0  0:00:04  0:00:03  0:00:01  192k
100  908k  100  908k    0     0   187k      0  0:00:04  0:00:04 --:--:--  187k
100  908k  100  908k    0     0   187k      0  0:00:04  0:00:04 --:--:--  205k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2  878k    2 25967    0     0  25967      0  0:0

In [3]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [5]:
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile = False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
                                           minSize = (30, 30))
    canvas = np.zeros((250, 300, 3), dtype = 'uint8')
    
    if len(faces) > 0:
        face = sorted(faces, reverse = True, key = lambda x: (x[2] - x[0]) * 
                     (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
        
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = '{} : {:.2f}%'.formate(emotion, prob * 100)
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
            
        cv2.imshow('Emotion Recognition', frame)
        cv2.imshow('Probabilities', canvas)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
camera.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.2) c:\miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


In [ ]:
https://m.blog.naver.com/roboholic84/221633210887